# h2o benchmarks - query 7

In [1]:
import coiled
import dask
import dask.dataframe as dd

In [2]:
cluster = coiled.Cluster(name="powers-crt-004", software="crt-004", n_workers=5)

Output()

In [4]:
client = dask.distributed.Client(cluster)

In [7]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet",
    storage_options={"anon": True, "use_ssl": True},
    columns=["id3", "v1", "v2"],
)

In [8]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

CPU times: user 1.26 s, sys: 267 ms, total: 1.53 s
Wall time: 1min 22s


,range_v1_v2
id3,
id0000608844,4
id0000466449,4
id0000573987,4
id0000776204,4
id0000608718,4
...,...
id0000821599,4
id0000937302,4
id0000248458,4
